In [11]:
def elves():
    with open("day1.txt") as f:
        data=[line.strip() for line in f.readlines()]
        def chunks():
            p = 0
            for d in data:
                if d=='':
                    yield p
                    p = 0
                else:
                    p += int(d)
        return chunks()
day1a = max(elves())

In [12]:
day1b = sum(sorted(elves(), reverse=True)[0:3])

In [13]:
day1a, day1b

(68923, 200044)

In [34]:
def day2b():
    with open("day2.txt") as f:
        guide=[line.strip() for line in f.readlines()]
        guide = list(map(lambda x: tuple(x.split(" ")), guide))
    
    win = {'A':'Y','B':'Z','C':'X'}
    lose = {'A':'Z','B':'X','C':'Y'}
    draw = {'A':'X','B':'Y','C':'Z'}
    scoring = {'X':1,'Y':2,'Z':3}

    score = lambda opp, you: scoring[you] + (6 if win[opp] == you else 3 if draw[opp] == you else 0)
    move = {'X':lose,'Y':draw,'Z':win}
    
    total_basic = sum([score(opp,you) for opp,you in guide])
    total_advanced = sum([score(opp,(move[you])[opp]) for opp,you in guide])

    return total_basic, total_advanced
day2b()

(11906, 11186)

In [64]:
def day3():
    with open("day3.txt") as f:
        rucksacks = [line.strip() for line in f.readlines()]
        make_set = lambda x: set(list(x))
        rucksacks = list(map(lambda x: (make_set(x[:len(x)//2]),make_set(x[len(x)//2:])), rucksacks))
    priority = lambda p: ord(p) - ord('a') + 1 if p.islower() else ord(p) - ord('A') + 27

    s = 0
    for l,r in rucksacks:
        common = l.intersection(r)
        s += priority(min(common))
    print(s)
    
    s = 0
    groups = [[rucksacks[i],rucksacks[i+1],rucksacks[i+2]] for i in range(0,len(rucksacks),3)]
    for group in groups:
        m = None
        for l,r in group:
            m = l.union(r) if m is None else m.intersection(l.union(r))
        s += priority(min(m))
    print(s)

day3()

7746
2604


In [73]:
def day4():
    with open("day4.txt") as f:
        cfg = [line.strip() for line in f.readlines()]
        inc_range = lambda x,y: range(int(x),int(y)+1)
        mk_range = lambda x: inc_range(*x.split('-'))
        cfg = list(map(lambda c: list(map(set,map(mk_range,c.split(",")))), cfg))
        
    def overlaps(c):
        left,right = tuple(c)
        return len(left.intersection(right)) > 0
    
    def contains(c):
        left,right = tuple(c)
        return len(left.intersection(right)) == min(len(left),len(right))
    
    a = sum(map(contains, cfg))
    b = sum(map(overlaps, cfg))

    print(a,b)
day4()

490 921


In [113]:
import re
def day5():
    with open("day5.txt") as f:
        lines = map(lambda l: l.strip(), f.readlines())
        crates = [[]] * 9
        crates_b = [[]] * 9
        for l in lines:
            for i,s in enumerate(l[1::4]):
                if s.isupper():
                    crates[i] = [s] + crates[i]
                    crates_b[i] = [s] + crates_b[i]
            if l == "":
                break
        pattern = re.compile("move (.*) from (.*) to (.*)")
        for l in lines:
            n, fr, to = map(int, pattern.search(l).groups())
            for _ in range(n):
                c = crates[fr-1].pop()
                crates[to-1].append(c)
            crates_b[to-1] = crates_b[to-1] + crates_b[fr-1][-n:]
            del crates_b[fr-1][-n:]
        print("".join([c[-1] for c in crates]))
        print("".join([c[-1] for c in crates_b]))

day5()

SBPQRSCDF
RGLVRCQSB


In [138]:
def day6(marker_length = 4):
    with open("day6.txt") as f:
        l = list(f.readline())
    for i in range(len(l)-marker_length):
        if len(set(l[i:i+marker_length])) == marker_length:
            print(i+marker_length)
            break
day6()
day6(14)

1282
3513


In [193]:
def day7():
    with open("day7.txt") as f:
        lines = map(lambda l: l.rstrip(), f.readlines())
    root = tree = {}
    last_tree = [root]
    for line in lines:
        if line[0] == "$":
            cmd = line[2:].split(" ")
            if cmd[0] == "cd":
                d = cmd[1]
                if d == "..":
                    tree = last_tree.pop()
                else:
                    last_tree.append(tree)
                    if d not in tree:
                        tree[d] = {}
                    tree = tree[d]
        else:
            info, file = line.split(" ")
            if info == "dir":
                tree[file] = {}
            else:
                tree[file] = int(info)
                
    def file_size(tree):
        total = 0
        for k,v in tree.items():
            if type(v) == int:
                total += v
            else:
                total += file_size(v)
        return total
    
    def small_dir_size(tree):
        total = 0
        for k,v in tree.items():
            if type(v) is not int:
                s = file_size(v)
                if s <= 100000:
                    total += s
                total += small_dir_size(v)
        return total
    
    print(small_dir_size(root))
    root_size = file_size(root)
    to_free = root_size - 4e7
    
    smallest = [None]
    def collect_smallest(tree, sm):
        for k,v in tree.items():
            if type(v) is not int:
                size = file_size(v)
                if size >= to_free and (sm[0] is None or sm[0] > (file_size(v)-to_free)):
                    sm[0] = file_size(v)-to_free
                collect_smallest(v, sm)
                
    collect_smallest(root, smallest)
    print(smallest[0]+to_free)
day7()
        

1743217
8319096.0


In [274]:
from collections import defaultdict
def day8():
    with open("day8.txt") as f:
        t = list(map(lambda l: list(l.rstrip()), f.readlines()))
    
    h = len(t)
    w = len(t[0])
    v = {}
    s = {}
    
    for y in range(h):
        for x in range(w):
            tree = t[y][x]
            los_x = [reversed(range(0,x)),range(x+1,w)]
            los_y = [reversed(range(0,y)),range(y+1,h)]
            scores = []
            
            tree_los = []
            
            for los in los_x:
                tree_los.append(list(map(lambda T: T < tree, [t[y][x] for x in los])))
            for los in los_y:
                tree_los.append(list(map(lambda T: T < tree, [t[y][x] for y in los])))
                
            for trees in tree_los:
                if all(trees):
                    v[(x,y)] = True
                    score = len(list(trees))
                else:
                    for idx,T in enumerate(trees):
                        if not T:
                            score = idx+1
                            break
                    
                scores.append(score)

            s[(x,y)] = scores[0]*scores[1]*scores[2]*scores[3]
            
    print(len(v.keys()))
    print(max(s.values()))
            
day8()
            

1662
537600


In [47]:
from PIL import Image, ImageDraw

def day9(N = 2):
    with open("day9.txt") as f:
        move = lambda move: {'R': (1,0), 'L': (-1,0), 'D': (0,1), 'U': (0,-1)}[move]
        tuples = list(map(lambda l: l.rstrip().split(" "), f.readlines()))
        moves = list(map(lambda t: (move(t[0]), int(t[1])), tuples))
    
    frames = []
    nodes = [[0,0] for i in range(N)]
    visited = set([(0,0)])
    for move in moves:
        for i in range(move[1]):
            head = nodes[0]
            head[0] += move[0][0]
            head[1] += move[0][1]

            for r in range(0,N-1):
                head = nodes[r]
                tail = nodes[r+1]
                
                if (tail[0] == head[0] and abs(tail[1]-head[1]) == 2):
                    tail[1]=head[1]+(1 if tail[1]>head[1] else -1)
                if (tail[1] == head[1] and abs(tail[0]-head[0]) == 2):
                    tail[0]=head[0]+(1 if tail[0]>head[0] else -1)
                if (tail[0] != head[0] and tail[1] != head[1] and not \
                    (abs(tail[0]-head[0]) == 1 and abs(tail[1]-head[1]) == 1)):
                    tail[0] += (1 if tail[0]<head[0] else -1)
                    tail[1] += (1 if tail[1]<head[1] else -1)
            visited.add(tuple(nodes[-1]))
            new_frame = Image.new("RGB", (300,300))
            draw = ImageDraw.Draw(new_frame)
            coords = list(map(lambda n: (n[0]*4+200, n[1]*4+100),nodes))
            draw.line(coords, fill="red", width=4)
            
        frames.append(new_frame)
    frames[0].save('rope.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)
    
    print(max(visited))
    print(min(visited))

    print(len(visited))
    
#day9()
day9(36)

(36, 25)
(-278, 119)
843


In [390]:
def day10():
    with open("day10.txt") as f:
        cmds = list(map(lambda l: l.rstrip().split(" "), f.readlines()))
        
    cycle = 0
    x = 1
    samples = []
    crt = [[] for _ in range(6)]
    
    def signal(cycle, samples, x):
        row = (cycle-1)//40
        char = "#" if abs(len(crt[row])-x) <= 1 else "."
        crt[row].append(char)
        if (cycle-20)%40 == 0:
            samples.append(x * cycle)
        
    for cmd in cmds:
        if cmd[0] == "noop":
            cycle += 1
            signal(cycle, samples, x)
            
        if cmd[0] == "addx":
            cycle += 1
            signal(cycle, samples, x)
            cycle += 1
            signal(cycle, samples, x)
            x += int(cmd[1])
        
    print(sum(samples))
    for row in crt:
        print("".join(row))
day10()

13860
###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..


In [457]:
def day11(a=True):
    
    m_re = re.compile("Monkey (.*):")
    i_re = re.compile(".*Starting items: (.*)")
    o_re = re.compile(".*Operation: (.*)")
    t_re = re.compile(".*Test: divisible by (.*)")
    on_re = re.compile(".*throw to monkey (.*)")

    monkeys = []
    gcd = 1
    activity = defaultdict(lambda: 0)
    
    with open("day11.txt") as f:
        while(True):
            monkey = int(m_re.search(f.readline().rstrip()).groups()[0])
            items = list(map(int, i_re.search(f.readline().rstrip()).groups()[0].split(",")))
            op = o_re.search(f.readline().rstrip()).groups()[0].split("=")[1]\
            op_as_func = eval('lambda old: ' + op)
            test = int(t_re.search(f.readline().rstrip()).groups()[0])
            gcd *= test
            on_true = int(on_re.search(f.readline().rstrip()).groups()[0])
            on_false = int(on_re.search(f.readline().rstrip()).groups()[0])
            blank = f.readline()
            monkeys.append({'idx':monkey, 'items':items,'op':op_as_func,'test':test,'on_true':on_true,'on_false':on_false})

            if not blank:
                break
    # one round
    for rnd in range(20 if a else 10000):
        for monkey in monkeys:
            N = len(monkey['items'])
            new = map(monkey['op'], monkey['items'])
            bored = list(map(lambda i: i//3 if a else i%gcd, new))
            div = map(lambda i: i%monkey['test'] == 0, bored)
            throw_to = map(lambda i: monkey['on_true'] if i else monkey['on_false'], div)

            monkey['items'] = []
            
            for t,b in zip(throw_to, bored):
                monkeys[t]['items'].append(b)
                
            activity[monkey['idx']] += N
    
    a,b = sorted(activity.values())[-2:]
    print(a*b)
day11(a=True)
day11(a=False)
            

50830
14399640002


In [587]:
def day12():
    with open("day12.txt") as f:
        m = list(map(lambda l: list(l.rstrip()), f.readlines()))
    start = [(ix,iy) for ix, row in enumerate(m) for iy, p in enumerate(row) if p == 'S'][0]
    end = [(ix,iy) for ix, row in enumerate(m) for iy, p in enumerate(row) if p == 'E'][0]

    H = len(m)
    W = len(m[0])
    
    m[start[0]][start[1]] = 'a'
    m[end[0]][end[1]] = 'z'
    
    current = start
    visited = []
        
    neighbours = lambda pos: [(pos[0]+d,pos[1]) for d in [-1, 1]] + [(pos[0],pos[1]+d) for d in [-1, 1]]
    reachable = lambda current, pos: pos[0] >= 0 and pos[0] < H and pos[1] >= 0 and pos[1] < W \
                        and ord(m[current[0]][current[1]]) >= ord(m[pos[0]][pos[1]])-1
    
    
    def bfs(start, end, deadends):
        queue = [(start,[start])]
        visited = set()

        while queue:
            vertex, path = queue.pop(0)
            visited.add(vertex)
            neigh = neighbours(vertex)
            reach = list(filter(lambda r: reachable(vertex, r) and r not in path and r not in deadends, neigh))

            if len(reach) == 0:
                deadends.add(vertex)
                
            for node in reach:
                if node == end:
                    return len(path)
                else:
                    if node not in visited:
                        visited.add(node)
                        queue.append((node, path + [node]))
    accum = []
    deadends = set()
    part1 = bfs(start, end, deadends)
    print(part1)
    
    starting_points = [(ix,iy) for ix, row in enumerate(m) for iy, p in enumerate(row) if p == 'a']
    starting_points = list(filter(lambda p: p not in deadends, starting_points))
    
    paths = map(lambda s: bfs(s, end, deadends), starting_points)
    part2 = min(p for p in paths if p is not None)
    print(part2)
    
day12()

440
439


In [64]:
import functools 

def day13():
    with open("day13.txt") as f:
        def cmp(left, right, verbose=False):
            if verbose:
                print(f"My comparing {left} with {right}")
            if type(left) is int and type(right) is int:
                if left == right:
                    return None
                return left < right
            if type(left) is int:
                return cmp([left], right, verbose=verbose)
            if type(right) is int:
                return cmp(left, [right], verbose=verbose)
            assert(type(left) is list and type(right) is list)
            
            for i,l in enumerate(left):
                if i >= len(right):
                    return False # Right ran out of items
                r = right[i]
                c = cmp(l, r, verbose=verbose)
                if c is None:
                    if verbose:
                        print("Continue ", left)
                    continue
                else:
                    return c
            return None if len(left) == len(right) else True # Left ran out of items

        i = 1
        s = 0
        
        packets = []
        while(True):
            line1 = f.readline().rstrip()
            line2 = f.readline().rstrip()
            l1 = eval(line1)
            l2 = eval(line2)
            packets += [l1]
            packets += [l2]
            c1 = cmp(l1, l2)

            if c1:
                s += i
            if c1 is None:
                print("Ooops")
                
            i += 1
            blank = f.readline()
            if not blank:
                break
        
        print(f"Part 1: {s}")
        
        packets += [[[2]]]
        packets += [[[6]]]
        
        cmp_wrapper = lambda l,r: -1 if cmp(l,r) else 1
        s = sorted(packets, key=functools.cmp_to_key(cmp_wrapper))
    
        print((s.index([[6]])+1)*(s.index([[2]])+1))
day13()

Part 1: 5529
27690


In [50]:
def day14(mode = "part1"):
    with open("day14.txt") as f:
        read_path = lambda l: list(map(lambda t: tuple(map(int, t.split(","))), l.split(" -> ")))
        paths = list(map(read_path, map(lambda l: l.rstrip(), f.readlines())))
        
    max_y = max_x = None
    for p in paths:
        for x,y in p:
            max_x = x if max_x is None else max(x, max_x)
            max_y = y if max_y is None else max(y, max_y)
    g = {}
    
    for path in paths:
        for p0,p1 in [(path[i], path[i+1]) for i in range(len(path)-1)]:
            rx = range(min(p0[0],p1[0]), max(p0[0],p1[0])+1)
            ry = range(min(p0[1],p1[1]), max(p0[1],p1[1])+1)
            
            for x,y in [(x,y) for x in rx for y in ry]:
                g[(x,y)] = "#"
    
    def pp(g):
        g_pp = []
        for y in range(max_y+2):
            g_pp.append(list("."*(max_x+1)))
            
        for x,y in g.keys():
            if x > 0 and x < max_x:
                g_pp[y][x] = g[(x,y)]
            
        for l in g_pp:
            print("".join(l[420:]))
    pp(g)

    # let the sand flow:
    src = (500,0)
    flowing = True
    done = False
    
    is_free = lambda t : t[1] <= max_y + 1 and t not in g
        
    frames = []
    grains = 0
    while (not done):
        sand = src
        while (True):
            if mode == "part1":
                if sand[1]+1 > max_y:
                    done = True
                    break
                    
            n_sand = (sand[0],sand[1]+1)
            nl_sand = (sand[0]-1,sand[1]+1)
            nr_sand = (sand[0]+1,sand[1]+1)

            free = list(filter(is_free, [n_sand, nl_sand, nr_sand]))
            
            if mode == "part2":
                if sand == (500,0) and len(free) == 0:
                    grains += 1
                    done = True
                    break
                    
            if len(free) > 0:
                sand = free[0]
            else:
                g[sand] = "o"
                
                if grains % 10 == 0:
                    new_frame = Image.new("RGB", (max_x,max_y), (50,50,40))
                    for x,y in g.keys():
                        if x > 0 and x < max_x and y > 0 and y < max_y:
                            color = (255,248,220) if g[(x,y)] == "#" else (255,222,173)
                            new_frame.putpixel((x,y), color)
                    frames.append(new_frame)
                grains += 1
                break
                
    
    if mode == "part2":
        frames[0].save('sand.gif', format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

    pp(g)
    print(grains)
day14()
day14(mode="part2")

...............................................................................................................
...............................................................................................................
...............................................................................................................
...............................................................................................................
...............................................................................................................
...............................................................................................................
...............................................................................................................
...............................................................................................................
........................................................................................................

...............................................................................................................
...............................................................................ooo.............................
..............................................................................ooooo............................
.............................................................................ooooooo...........................
............................................................................ooooooooo..........................
...........................................................................ooooooooooo.........................
..........................................................................ooooooooooooo........................
.........................................................................ooooooooooooooo.......................
........................................................................ooooooooooooooooo...............

In [95]:
list(range(2,1,-1))

[2]

In [104]:
list(range(500,500))

[]